In [ ]:
#import modules

from splinter import Browser
from bs4 import BeautifulSoup 
from urllib.parse import urljoin
import pymongo
import requests
import pandas as pd
from sqlalchemy import create_engine


In [ ]:
#opening chromedriver browser
browser = Browser('chrome')
url = 'insert url here'

# get the details about the author from author page 

In [ ]:
#this functions takes the url from the author page, scrpae the information from the page and return
#a dictionary

def get_author(url):
    results = {}
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    result['born'] = soup.find('span', class_='author-born-date').text.strip()
    result['name'] = soup.h3.text.strip()
    result['description'] = soup.find('div', class_='author-description').text.strip()
    return result

In [ ]:
def get_tags_for_quote(quote_soup):
    tags=[]
    for rag in quote_soup.find_all('a',class_='tag'):
        tags.append(tag.text)
    return tags

# get all the details about one quote 

In [ ]:
def get_quote(quote_soup):
    quote = ()
    quote['text'] = quote_soup.span.text
    quote['author'] = quote_soup.find('small',class_='author').text
    href=quote_soup.a['href']
    author_url = urljoin(url,href)
    #for this quote get the author details now
    quote['author']=get_author(author_url)
    quote['tags']=get_tags_for_quote(quote_soup)
    return quote

In [ ]:
def get_quotes_on_a_page(page_number):
    result=[]
    html=browser.html
    soup=BeautifulSoup(html,'lxml')
    quote=soup.find_all('div',class_='quote')
    number_of_quotes_per_page=10
    quote_id=(page_number-1)*number_of_quotes_per_page
    for quote in quotes:
        quote_id = quote_id + 1
        quote_info =get_quote(quote)
        quote_info['_id'] = quote_id
        result.append(quote_info)
    return result

In [ ]:
def scrape_all_quotes(url):
    more_quotes =True
    first_iterations = True
    page_number = 0
    quotes = []
    while more_quotes:
        page_number += 1
        print(f'Now scrapping page : {page_number}')
        if first_iterations:
            browser.visit(url)
            first_iterations =False
        else:
            pass:
        quote_on_this_page = get_quotes_on_a_page(page_number)
        quotes = quotes + quotes_on_this_page
        try:
            next = browser.links.find_by_partial_text('Next')
            print('about to click on the next link')
            if(next.is_empty()):
                more_quotes=False
            else:
                next.click()
        except Exception as ex:
            print('Scraping Complete')
            print(ex.message)
            more_quotes = False
    return quotes